In [82]:
import re, sys
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
import warnings
from rdkit import Chem, DataStructs, RDLogger
from rdkit.Chem import rdChemReactions, AllChem, Draw, PandasTools
from torch.utils.data import Dataset, DataLoader

RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Transform

In [71]:
df_mols = pd.read_pickle("../data/transformed/df_mols_ready.pkl")

In [72]:
df_mols['bbs_smi'] = df_mols['A_smi'] + '.' + df_mols['C_smi'] + '.' + df_mols['C_smi']

#### Encode/decode

In [76]:
pad_id = 0  # assuming pad token is 0

def encode(smiles, stoi, max_len=128):
    # returns list of token IDs with <s> at start, </s> at end, padded
    ids = [stoi["<s>"]] + [stoi[ch] for ch in smiles] + [stoi["</s>"]]
    if len(ids) < max_len:
        ids += [stoi["<pad>"]] * (max_len - len(ids))
    return ids[:max_len]

def decode(ids, itos):
    # returns string ignoring special tokens
    chars = [itos[i] for i in ids if itos[i] not in ("<pad>", "<s>", "</s>")]
    return "".join(chars)

In [77]:
def encode_target(smi, stoi, max_len=128):
    full_ids = encode(smi, stoi, max_len)   # <s> ... </s> + pad
    tgt_in = full_ids[:-1]                  # decoder input (drop final </s>)
    tgt_out = full_ids[1:]                  # loss target (drop initial <s>)
    return tgt_in, tgt_out

# Apply to dataframe
df_mols['tgt_in'], df_mols['tgt_out'] = zip(*df_mols['bbs_smi'].apply(lambda x: encode_target(x, stoi)))

In [ ]:
def decode_show_special(ids, itos):
    chars = [itos[i] for i in ids]  # do NOT filter
    return "".join(chars)

print("Decoder input with specials:", decode_show_special(sample_in, itos))
print("Target for loss with specials:", decode_show_special(sample_out, itos))


In [95]:
class MolDataset(Dataset):
    def __init__(self, df, stoi, max_len=128):
        self.src = [encode(s, stoi, max_len) for s in df['mol_smi']]
        self.tgt_in = df['tgt_in'].tolist()
        self.tgt_out = df['tgt_out'].tolist()
    def __len__(self):
        return len(self.src)
    def __getitem__(self, idx):
        return torch.tensor(self.src[idx]), torch.tensor(self.tgt_in[idx]), torch.tensor(self.tgt_out[idx])

In [97]:
# Step 2a: Split dataframe into train/validation
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_mols, test_size=0.2, random_state=42)

# Step 3: Create datasets
train_dataset = MolDataset(df_train, stoi)
val_dataset   = MolDataset(df_val, stoi)

# Step 4: Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)

In [84]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model=384, n_layers=2, dropout=0.1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.rnn = nn.GRU(d_model, d_model, num_layers=n_layers, batch_first=True, dropout=dropout)
    def forward(self, x):
        x = self.emb(x)
        out, h = self.rnn(x)
        return out, h

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model=384, n_layers=2, dropout=0.1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.rnn = nn.GRU(d_model, d_model, num_layers=n_layers, batch_first=True, dropout=dropout)
        self.proj = nn.Linear(d_model, vocab_size)
    def forward(self, y_in, h):
        y = self.emb(y_in)
        out, h = self.rnn(y, h)
        logits = self.proj(out)
        return logits, h

class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model=384, n_layers=2, dropout=0.1):
        super().__init__()
        self.enc = Encoder(vocab_size, d_model, n_layers, dropout)
        self.dec = Decoder(vocab_size, d_model, n_layers, dropout)
    def forward(self, src, tgt_in):
        _, h = self.enc(src)
        logits, _ = self.dec(tgt_in, h)
        return logits


In [85]:
vocab_size = len(stoi)
model = Seq2Seq(vocab_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)  # ignore padding in loss

In [88]:
n_epochs = 30
checkpoint_path = "seq2seq_gru_bbs.pt"

# --- Resume from checkpoint if exists
start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming from epoch {start_epoch}")

Resuming from epoch 30


In [98]:
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt_in, tgt_out in loader:
            src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)
            logits = model(src, tgt_in)
            loss = criterion(logits.view(-1, logits.size(-1)), tgt_out.view(-1))
            total_loss += loss.item() * src.size(0)
    return total_loss / len(loader.dataset)

In [99]:
start_epoch = 0

In [100]:
# --- Training loop
for epoch in range(start_epoch, n_epochs):
    model.train()
    total_train_loss = 0

    for src, tgt_in, tgt_out in train_loader:
        src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)
        optimizer.zero_grad()
        logits = model(src, tgt_in)
        loss = criterion(logits.view(-1, logits.size(-1)), tgt_out.view(-1))
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * src.size(0)

    train_loss = total_train_loss / len(train_loader.dataset)
    
    # --- Compute validation loss
    val_loss = validate(model, val_loader, criterion, device)
    
    # --- Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)
    
    print(f"Epoch {epoch:02d} | train loss {train_loss:.4f} | val loss {val_loss:.4f}")

Epoch 00 | train loss 0.0354 | val loss 0.0351
Epoch 01 | train loss 0.0352 | val loss 0.0425
Epoch 02 | train loss 0.0339 | val loss 0.0429
Epoch 03 | train loss 0.0346 | val loss 0.0388
Epoch 04 | train loss 0.0323 | val loss 0.0411
Epoch 05 | train loss 0.0329 | val loss 0.0408
Epoch 06 | train loss 0.0322 | val loss 0.0411
Epoch 07 | train loss 0.0310 | val loss 0.0464
Epoch 08 | train loss 0.0311 | val loss 0.0402
Epoch 09 | train loss 0.0277 | val loss 0.0408
Epoch 10 | train loss 0.0272 | val loss 0.0409
Epoch 11 | train loss 0.0356 | val loss 0.0444
Epoch 12 | train loss 0.0301 | val loss 0.0432
Epoch 13 | train loss 0.0281 | val loss 0.0425
Epoch 14 | train loss 0.0259 | val loss 0.0374
Epoch 15 | train loss 0.0258 | val loss 0.0394
Epoch 16 | train loss 0.0252 | val loss 0.0431
Epoch 17 | train loss 0.0273 | val loss 0.0407
Epoch 18 | train loss 0.0270 | val loss 0.0405
Epoch 19 | train loss 0.0259 | val loss 0.0409
Epoch 20 | train loss 0.0244 | val loss 0.0400
Epoch 21 | tr

In [101]:
def predict_bbs(model, src_smiles, stoi, itos, max_len=128, device=device):
    model.eval()
    src_ids = torch.tensor([encode(src_smiles, stoi, max_len)]).to(device)
    
    # --- Encode once
    _, h = model.enc(src_ids)
    
    # --- Decoder starts with <s>
    tgt_ids = torch.tensor([[stoi["<s>"]]], device=device)
    pred_tokens = []

    for _ in range(max_len):
        logits, h = model.dec(tgt_ids, h)
        next_token = logits[:, -1, :].argmax(-1, keepdim=True)
        if next_token.item() == stoi["</s>"]:
            break
        pred_tokens.append(next_token.item())
        tgt_ids = torch.cat([tgt_ids, next_token], dim=1)

    pred_bbs = "".join([itos[i] for i in pred_tokens])
    return pred_bbs

In [102]:
for i in range(3):
    mol_smi = df_mols['mol_smi'].iloc[i]
    true_bbs = df_mols['bbs_smi'].iloc[i]
    pred_bbs = predict_bbs(model, mol_smi, stoi, itos)
    print(f"MOL  : {mol_smi}")
    print(f"TRUE : {true_bbs}")
    print(f"PRED : {pred_bbs}\n")

MOL  : COc1cccc(OC)c1[C@@H]1C[C@H](F)C(=O)N1Cc1ccc(OC(F)(F)F)cc1
TRUE : [*]c1c(OC)cccc1OC.[*]Cc1ccc(OC(F)(F)F)cc1.[*]Cc1ccc(OC(F)(F)F)cc1
PRED : [*]c1c(OC)cccc1OC.[*]Cc1ccc(OC(F)(F)C(F)F)cc1.[*]Cc1ccc(OC(F)(F)C(F)F)cc1

MOL  : COc1cccc(OC)c1[C@@H]1CC(F)(F)C(=O)N1Cc1ccc(OC(F)(F)F)cc1
TRUE : [*]c1c(OC)cccc1OC.[*]Cc1ccc(OC(F)(F)F)cc1.[*]Cc1ccc(OC(F)(F)F)cc1
PRED : [*]c1c(OC)cccc1C(F)(F)F.[*]Cc1ccc(OC(F)(F)F)cc1.[*]Cc1ccc(OC(F)(F)F)cc1

MOL  : COc1cccc(OC)c1[C@@H]1CCC(=O)N1Cc1ccc(OC(F)(F)F)cc1
TRUE : [*]c1c(OC)cccc1OC.[*]Cc1ccc(OC(F)(F)F)cc1.[*]Cc1ccc(OC(F)(F)F)cc1
PRED : [*]c1c(OC)cccc1OC.[*]Cc1ccc(OC(F)(F)C(F)F)cc1.[*]Cc1ccc(OC(F)(F)C(F)F)cc1

